In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from hotcoding import *
from postprocessing import *

In [2]:
np.random.seed(1000)

In [3]:
TF_SNP_count = pd.read_csv("../../../data/product/tf_snp_count.csv")

In [4]:
expression_matrix = pd.read_csv("../../../data/expression_matrix.txt", delimiter='\t')

In [5]:
target_expression = pd.read_csv("../../../data/product/expression_matrix_tf_targets_only.csv", header=0)
target_expression_matrix = target_expression.set_index('CONDITION')

In [6]:
TF = TF_SNP_count.iloc[0:49,]

In [7]:
TF_expression_matrix = expression_matrix.iloc[TF['ID'],].T.drop('ID').apply(lambda x: x.fillna(x.mean()), axis=0)

In [8]:
TF_RF = RandomForestRegressor(max_depth=10, n_estimators=2000, n_jobs=-1)

In [9]:
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(TF_expression_matrix)

10

In [10]:
RF_score = []
for train_index, test_index in kf.split(TF_expression_matrix):
    X_train, X_test = TF_expression_matrix.iloc[train_index], TF_expression_matrix.iloc[test_index]
    y_train, y_test = target_expression_matrix.iloc[train_index], target_expression_matrix.iloc[test_index]
    TF_RF.fit(X_train, y_train)
    print(TF_RF.score(X_test,y_test))
    RF_score.append(TF_RF.score(X_test,y_test))


0.13892833944804905
0.0676080659048998
0.08347072886829905
0.19142596811339033
0.11121228728131484
0.03524025145010454
0.1956270723847565
0.09779287258389661
-0.004135876497003695
0.1077083321818628


In [11]:
avg_TF_score = sum(RF_score)/10
avg_TF_score

0.10248780417195698

In [12]:
# PERFORM ONE HOT ENCODING OF SNPS

In [13]:
# creates a one hot encoding of all the unique SNPs within TFs
one_hotcode_genotype_only_unique_snps()

,SNP_897_0,SNP_897_1,SNP_1546_0,SNP_1546_1,SNP_1546_2,SNP_1547_0,SNP_1547_1,SNP_1547_2,SNP_25_0,SNP_25_1,...,SNP_2297_2,SNP_251_0,SNP_251_1,SNP_251_2,SNP_252_0,SNP_252_1,SNP_252_2,SNP_253_0,SNP_253_1,SNP_253_2
CONDITION,,,,,,,,,,,,,,,,,,,,,
21_5_c,0,1,1,0,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
22_2_d,1,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
19_2_c,0,1,0,1,0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0
19_3_c,1,0,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
19_4_b,1,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
19_5_b,1,0,1,0,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
20_1_d,0,1,1,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
20_2_d,1,0,1,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
20_3_c,0,1,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0


In [14]:
# Train new regressor with unique SNPs within TFs added as input
#
#

In [15]:
np.random.seed(1000)

In [16]:
matrix_genotypes_hotcoded_snps = pd.read_csv("../../../data/product/matrix_genotypes_hotcoded_snps_collapsed.csv")

In [17]:
genotypes_hotcoded_snps = matrix_genotypes_hotcoded_snps.set_index('CONDITION')

In [18]:
TF_SNP = pd.concat([TF_expression_matrix, genotypes_hotcoded_snps], axis=1).reindex(TF_expression_matrix.index)

In [19]:
TF_SNP_RF = RandomForestRegressor(max_depth=10, n_estimators=2000, n_jobs=-1)

In [20]:
#kf2 = KFold(n_splits=10, shuffle=True)
#kf2.get_n_splits(TF_SNP)
kf2 = kf

In [21]:
TF_SNP_RF_score = []

importances = []
for train_index2, test_index2 in kf2.split(TF_SNP):
    X_train2, X_test2 = TF_SNP.iloc[train_index2], TF_SNP.iloc[test_index2]
    y_train2, y_test2 = target_expression_matrix.iloc[train_index2], target_expression_matrix.iloc[test_index2]
    TF_SNP_RF.fit(X_train2, y_train2)
    print(TF_SNP_RF.score(X_test2,y_test2))
    TF_SNP_RF_score.append(TF_SNP_RF.score(X_test2,y_test2))
    importances.append(TF_SNP_RF.feature_importances_)

0.1368271485167555
0.05987446286995909
0.08179780461217086
0.19124264394989682
0.11145326674745165
0.03222471137752721
0.1899897301153807
0.09897715232635064
0.0002986134459712396
0.09992505040415706


In [23]:
avg_TF_SNP_score = sum(TF_SNP_RF_score)/10
avg_TF_SNP_score

0.10026105843656208

In [24]:
imp_df = pd.DataFrame(data=importances, columns=TF_SNP.columns.values)

imp_df.to_csv('../../../results/rf_importances.csv', index=False)
imp_df

,5513,78,4414,336,4442,64,1677,2394,2283,1925,...,SNP_2297_2,SNP_251_0,SNP_251_1,SNP_251_2,SNP_252_0,SNP_252_1,SNP_252_2,SNP_253_0,SNP_253_1,SNP_253_2
0,0.012590,0.012566,0.014861,0.011840,0.014742,0.013591,0.011597,0.010635,0.012693,0.015452,...,0.000536,0.001855,0.001862,0.000000,0.001869,0.001911,0.000000,0.001863,0.001916,0.000226
1,0.013369,0.012866,0.020095,0.011067,0.014212,0.012211,0.012736,0.011979,0.013061,0.015457,...,0.000288,0.001808,0.001783,0.000061,0.002033,0.002063,0.000048,0.001654,0.002017,0.000146
2,0.013309,0.012527,0.015810,0.012756,0.014444,0.014218,0.012540,0.010342,0.014397,0.014978,...,0.000687,0.002078,0.001793,0.000043,0.001808,0.001694,0.000016,0.001871,0.001743,0.000272
3,0.012998,0.013076,0.015088,0.012866,0.015524,0.013504,0.014425,0.010863,0.014129,0.012888,...,0.000495,0.001978,0.001844,0.000071,0.001926,0.001804,0.000071,0.001808,0.002086,0.000226
4,0.009828,0.013404,0.014563,0.012251,0.013818,0.013717,0.012042,0.011870,0.012490,0.015305,...,0.000596,0.001966,0.001902,0.000057,0.001759,0.001753,0.000049,0.001811,0.002047,0.000160
5,0.011915,0.012408,0.014573,0.011966,0.014759,0.011660,0.013592,0.011272,0.012405,0.013051,...,0.000733,0.001647,0.001727,0.000055,0.001987,0.001639,0.000050,0.001863,0.001779,0.000197
6,0.013830,0.013357,0.016123,0.013930,0.013338,0.013319,0.010801,0.012656,0.013237,0.013072,...,0.000715,0.001607,0.001952,0.000043,0.002047,0.001942,0.000046,0.001938,0.002046,0.000146
7,0.017563,0.012720,0.016526,0.012700,0.015389,0.013851,0.012105,0.010464,0.012770,0.015939,...,0.000934,0.001750,0.001571,0.000036,0.001673,0.001923,0.000039,0.002068,0.001999,0.000237
8,0.010195,0.011999,0.014741,0.012460,0.014567,0.011562,0.009893,0.011023,0.013169,0.014394,...,0.000757,0.001720,0.001854,0.000044,0.001919,0.002064,0.000061,0.002043,0.001665,0.000086
9,0.015561,0.012525,0.013959,0.011590,0.015257,0.013978,0.011460,0.012039,0.012258,0.013924,...,0.000695,0.002085,0.001886,0.000034,0.002074,0.001718,0.000051,0.001781,0.001895,0.000199


In [25]:
imp_df = sort_importance_table(imp_df)
imp_df

SNP_252_2     0.000043
SNP_251_2     0.000044
SNP_2220_2    0.000045
SNP_2481_2    0.000048
SNP_1547_2    0.000074
SNP_2607_2    0.000094
SNP_1546_2    0.000114
SNP_2516_2    0.000119
SNP_68_2      0.000123
SNP_2599_2    0.000136
SNP_2222_2    0.000159
SNP_2417_2    0.000167
SNP_2464_2    0.000175
SNP_253_2     0.000190
SNP_2223_2    0.000251
SNP_70_2      0.000274
SNP_2216_2    0.000436
SNP_2482_2    0.000447
SNP_81_2      0.000468
SNP_2217_2    0.000516
SNP_740_2     0.000558
SNP_2219_2    0.000587
SNP_2463_2    0.000624
SNP_2297_2    0.000644
SNP_2295_2    0.000656
SNP_2296_2    0.001177
SNP_2216_0    0.001257
SNP_2217_0    0.001258
SNP_2223_1    0.001260
SNP_2219_0    0.001297
                ...   
2325          0.012839
1842          0.012851
5118          0.013008
2283          0.013061
5513          0.013116
64            0.013161
5391          0.013347
3443          0.013489
5133          0.013573
5416          0.013627
983           0.013904
897           0.014024
1925       

In [28]:
# consider only the 9 highest ranking SNPs
imp_df = imp_df[[x for x in imp_df.index.values if str(x).startswith('SNP')]].tail(9)

In [29]:
most_important_snp_ids = [int(x[4:]) for x in imp_df.index.values]
most_important_snp_ids

[2295, 2297, 740, 81, 2296, 1643, 251, 68, 25]

In [30]:
# ONE HOT ENCODE ONLY THE MOST IMPORTANT SNPS
one_hotcode_genotype_only_important(most_important_snp_ids)

,SNP_2295_0,SNP_2295_1,SNP_2295_2,SNP_2297_0,SNP_2297_1,SNP_2297_2,SNP_740_0,SNP_740_1,SNP_740_2,SNP_81_0,...,SNP_1643_1,SNP_1643_2,SNP_251_0,SNP_251_1,SNP_251_2,SNP_68_0,SNP_68_1,SNP_68_2,SNP_25_0,SNP_25_1
CONDITION,,,,,,,,,,,,,,,,,,,,,
21_5_c,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
22_2_d,0,1,0,0,1,0,1,0,0,1,...,1,0,1,0,0,1,0,0,1,0
19_2_c,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,1,0,0,1,0
19_3_c,0,1,0,0,1,0,0,1,0,0,...,0,1,1,0,0,1,0,0,1,0
19_4_b,1,0,0,1,0,0,0,1,0,1,...,0,0,0,1,0,1,0,0,1,0
19_5_b,1,0,0,1,0,0,1,0,0,0,...,0,1,1,0,0,0,1,0,0,1
20_1_d,0,1,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,1
20_2_d,1,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
20_3_c,1,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1


In [31]:
# Train new regressor with only SNPs found to be important (25) within TFs added as input
#
#

In [32]:
np.random.seed(1000)

In [33]:
genotypes_hotcoded_snps_important = pd.read_csv("../../../data/product/matrix_genotypes_hotcoded_snps_only_important.csv").set_index('CONDITION')

In [34]:
TF_SNP_important = pd.concat([TF_expression_matrix, genotypes_hotcoded_snps_important], axis=1).reindex(TF_expression_matrix.index)

In [35]:
TF_SNP_important_RF = RandomForestRegressor(max_depth=10, n_estimators=2000, n_jobs=-1)

In [36]:
#kf3 = KFold(n_splits=10, shuffle=True)
#kf3.get_n_splits(TF_SNP_important)
kf3=kf

In [37]:
TF_SNP_important_RF_score = []

for train_index3, test_index3 in kf3.split(TF_SNP_important):
    X_train3, X_test3 = TF_SNP_important.iloc[train_index3], TF_SNP_important.iloc[test_index3]
    y_train3, y_test3 = target_expression_matrix.iloc[train_index3], target_expression_matrix.iloc[test_index3]
    TF_SNP_important_RF.fit(X_train3, y_train3)
    print(TF_SNP_important_RF.score(X_test3,y_test3))
    TF_SNP_important_RF_score.append(TF_SNP_important_RF.score(X_test3,y_test3))

0.1370902531602956
0.06299245095474781
0.08373125512819074
0.1921916144114916
0.11247614597930695
0.03147564042213219
0.19202463430582367
0.09395942317745144
0.0024594292268511654
0.10127220878779164


In [38]:
avg_TF_SNP_important_score = sum(TF_SNP_important_RF_score)/10
avg_TF_SNP_important_score

0.1009673055554083

In [44]:
df = pd.DataFrame(data=[avg_TF_score, avg_TF_SNP_score, avg_TF_SNP_important_score], columns=['rf_test_score'])
df.index = ['TF_only', 'TF_all_SNPs', 'TF_important_SNPs']
df.to_csv('../../../results/rf_training_scores.csv', index=False)
df

,rf_test_score
TF_only,0.102488
TF_all_SNPs,0.100261
TF_important_SNPs,0.100967


,rf_test_score
TF_only,0.102488
TF_all_SNPs,0.100261
TF_important_SNPs,0.100967
